# Processing

## Imports

In [8]:
%pip install scipy
import importlib as IL
import CU_POLARIS_Postprocessor
from CU_POLARIS_Postprocessor.config import PostProcessingConfig
from CU_POLARIS_Postprocessor import prerun, parallel
import CU_POLARIS_Postprocessor.parallel
import CU_POLARIS_Postprocessor.postprocessing
from CU_POLARIS_Postprocessor.power_bi_processing import prep_utils
from pathlib import Path
import importlib as IL
import CU_POLARIS_Postprocessor
import CU_POLARIS_Postprocessor.power_bi_processing
import CU_POLARIS_Postprocessor.power_bi_processing.prep_utils
import CU_POLARIS_Postprocessor.power_bi_processing.charting as pbi_charting
IL.reload(CU_POLARIS_Postprocessor.power_bi_processing.prep_utils)
IL.reload(CU_POLARIS_Postprocessor.postprocessing)
IL.reload(CU_POLARIS_Postprocessor.parallel)




Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


<module 'CU_POLARIS_Postprocessor.parallel' from 'c:\\polaris\\CU_POLARIS_Postprocessor\\CU_POLARIS_Postprocessor\\parallel.py'>

## Config

In [4]:
config_wtp=PostProcessingConfig(base_dir=Path("C:\\polaris\\data\\tests_local\\initial_tests"),
                                output_h5=True,
                                fresh_start=False,
                                reset_csvs=True,
                                reset_sql=True,
                                reset_stops=False, 
                                db_names=['greenville'],
                                scenario_file_names=['scenario_abm_jar.json','scenario_abm_default_repo.json','scenario_abm_no_repo.json'],
                                fleet_model_file_names=['config\\TNCFleetModel_joint_proactive_repo.json','config\\TNCFleetModel_proactive_default_repo.json','config\\TNCFleetModel_proactive_no_repo.json'],)

## Preprocessing

In [9]:
prerun.pre_run_checks(config_wtp)


True

### Show output of prerun

In [10]:
import pandas as pd
csvs = config_wtp.csvs
demo = []
for key, value in csvs.items():
        # Extract the values for each column
        type_val = value.get('type', None)
        exists_val = value.get('exists', None)
        location_val = value.get('path', None)
        
        # Append the row as a tuple
        demo.append((key, type_val, exists_val, location_val))

    # Create a DataFrame from the rows
df = pd.DataFrame(demo, columns=['Key', 'Type', 'Exists', 'Location'])
df

,Key,Type,Exists,Location
0,transit_trip_max_load_helper,sql,False,C:\polaris\data\tests_local\initial_tests\tran...
1,transit_trip_max_load,sql,False,C:\polaris\data\tests_local\initial_tests\tran...
2,mode_Distribution_ADULT_Counts,sql,False,C:\polaris\data\tests_local\initial_tests\mode...
3,mode_Distribution_ADULT_Distance,sql,False,C:\polaris\data\tests_local\initial_tests\mode...
4,bus_avo,sql,False,C:\polaris\data\tests_local\initial_tests\bus_...
5,pr_avo,sql,False,C:\polaris\data\tests_local\initial_tests\pr_a...
6,fare_sensitivity_results,sql,False,C:\polaris\data\tests_local\initial_tests\fare...
7,mode_Distribution_ADULT,sql,False,C:\polaris\data\tests_local\initial_tests\mode...
8,distance_tnc_dist,sql,False,C:\polaris\data\tests_local\initial_tests\dist...
9,fare_sensitivity_results_zonal,sql,False,C:\polaris\data\tests_local\initial_tests\fare...


## Process

This takes about 10 minutes for these output folders from scratch, but it builds on existing incomplete processing or pulls results as needed.

In [11]:
parallel.parallel_process_folders(config_wtp)


starting on directory C:\polaris\data\tests_local\initial_tests\gsc_300_dr
starting on directory C:\polaris\data\tests_local\initial_tests\gsc_300_jar
starting on directory C:\polaris\data\tests_local\initial_tests\gsc_300_nr


UnboundLocalError: cannot access local variable 'db_name' where it is not associated with a value

### Show Output of processing

In [6]:
for key, value in config_wtp.results.items():
    print(key)

transit_trip_max_load_helper
transit_trip_max_load
mode_Distribution_ADULT_Counts
mode_Distribution_ADULT_Distance
bus_avo
pr_avo
fare_sensitivity_results
mode_Distribution_ADULT
distance_tnc_dist
fare_sensitivity_results_zonal
fare_sensitivity_demograpic_tnc_stats
fare_sensitivity_results_vo
tnc_results_discount
elder_demo
activity_times
requests_sum
closest_stops
tnc_stat_summary
tnc_skim_demo
demo_financial_case_data


### Power BI Post Processing

In [7]:
IL.reload(CU_POLARIS_Postprocessor.power_bi_processing.prep_utils)
#use this for comparing all cases (within a city model) to a single base case
base_case_folders = ['atx_du_7','gvl_du_7']
#prep_utils.process_tnc_ttests(config=config_wtp,base_cases=base_case_folders)

#Use this for comparing strategy to strategy
#prep_utils.process_tnc_ttests(config=config_wtp,base_suffix="_heur")

#Use this for folder names structured cit_fleet-size_strategy_iteration
prep_utils.process_folder_names_cit_fleet_strat(config_wtp)

#Use this for folder names structured cit_case_iteration
#prep_utils.process_folder_names_wtp(config_wtp)

case_aggregators =["City","Strategy"]
demo_aggregators =["age_class"]
folder_to_columns = ["City","Strategy","Iteration","Fleet Size"]
prep_utils.generate_pbix_control_csv(config_wtp,case_aggregators,demo_aggregators,folder_to_columns)
prep_utils.update_h5(config=config_wtp)

Creating case columns for transit_trip_max_load_helper.
Creating case columns for transit_trip_max_load.
Creating case columns for mode_Distribution_ADULT_Counts.
Creating case columns for mode_Distribution_ADULT_Distance.
Creating case columns for bus_avo.
Creating case columns for pr_avo.
Creating case columns for fare_sensitivity_results.
Creating case columns for mode_Distribution_ADULT.
Creating case columns for distance_tnc_dist.
Creating case columns for fare_sensitivity_results_zonal.
Creating case columns for fare_sensitivity_demograpic_tnc_stats.
Creating case columns for fare_sensitivity_results_vo.
Creating case columns for tnc_results_discount.
Creating case columns for elder_demo.
Creating case columns for activity_times.
Creating case columns for requests_sum.
Creating case columns for closest_stops.
Creating case columns for tnc_stat_summary.
Creating case columns for tnc_skim_demo.
Creating case columns for demo_financial_case_data.


c:\polaris\CU_POLARIS_Postprocessor\CU_POLARIS_Postprocessor\power_bi_processing\prep_utils.py:170: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['mode', 'type', 'folder', 'City', 'Fleet Size', 'Strategy',
       'Iteration'],
      dtype='object')]

  store[key] = df
c:\polaris\CU_POLARIS_Postprocessor\CU_POLARIS_Postprocessor\power_bi_processing\prep_utils.py:170: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['Case Aggregators', 'Folder to Columns', 'Demographic Aggregators'], dtype='object')]

  store[key] = df


True

### Table Aggregation and Reformatting

In [11]:
#reload the pbi_charting module to ensure it has the latest changes
IL.reload(CU_POLARIS_Postprocessor.power_bi_processing.charting)
pbix_aggregators  = {
    'Transit Aggregators': ['City', 'Strategy'],
    'Folder to Columns': ['City', 'Strategy','Fleet Size'],
    'Demographic Aggregators':['City','age_class'],
    'Case Aggregators': ['City', 'Fleet Size'],
    'Strategy Aggregators': ['City', 'Strategy']
}
work_path = config_wtp.base_dir
#db_name_folder_map = {
#    'greenville':'gsc',
#    'campo':'atx'
#}
db_name_folder_map = {
    'Bloomington':'bloomington'
}
study_name = 'TRB_Case_Study'
#generates a new h5 file called pbix_tables.h5 which are directly used to generate charts in power bi
pbi_charting.run_all(work_path,pbix_aggregators,db_name_folder_map,study_name,db_path={"Bloomington":r"C:\polaris\data\for_processing\bloomington_1200\bloomington_iteration_1\Bloomington-Supply.sqlite"})
#pbi_charting.run_all(work_path,pbix_aggregators,db_name_folder_map,study_name,db_path={"greenville":r"C:\polaris\data\for_processing\gsc_100\greenville-Supply.sqlite"})




Processing:  38%|███▊      | 3/8 [00:00<00:00, 25.13step/s] 

Searching for supply databases.
Running transit queries.
Running mode shift queries.
Running demographic queries.


Processing:  88%|████████▊ | 7/8 [00:00<00:00, 15.40step/s]c:\polaris\CU_POLARIS_Postprocessor\CU_POLARIS_Postprocessor\power_bi_processing\charting.py:1645: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block2_values] [items->Index(['City', 'Strategy', 'Fleet Size', 'type', 'TNC Time Budget Level',
       'Bus Time Budget Level', 'Rail Time Budget Level'],
      dtype='object')]

  store[key] = df
c:\polaris\CU_POLARIS_Postprocessor\CU_POLARIS_Postprocessor\power_bi_processing\charting.py:1645: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block2_values] [items->Index(['City', 'city_zone'], dtype='object')]

  store[key] = df
Processing: 100%|██████████| 8/8 [00:00<00:00, 15.71step/s]

Running financial queries.
Running t-test queries.
Copying PBIX into case folder.
Creating h5 file.


### Open the PBIX

In [12]:
pbi_charting.open_pbix_file(config_wtp.base_dir,study_name)

### Load in final results

In [12]:
import pandas as pd
h5_path = config_wtp.base_dir.as_posix()+'/results.h5'

mode_Distribution_ADULT_Distance=  pd.read_hdf(h5_path,key='mode_Distribution_ADULT_Distance')
mode_Distribution_ADULT_Counts=  pd.read_hdf(h5_path,key='mode_Distribution_ADULT_Counts')
bus_avo=  pd.read_hdf(h5_path,key='bus_avo')
pr_avo=  pd.read_hdf(h5_path,key='pr_avo')
fare_sensitivity_results=  pd.read_hdf(h5_path,key='fare_sensitivity_results')
mode_Distribution_ADULT=  pd.read_hdf(h5_path,key='mode_Distribution_ADULT')
distance_tnc_dist=  pd.read_hdf(h5_path,key='distance_tnc_dist')
fare_sensitivity_results_zonal=  pd.read_hdf(h5_path,key='fare_sensitivity_results_zonal')
fare_sensitivity_demograpic_tnc_stats=  pd.read_hdf(h5_path,key='fare_sensitivity_demograpic_tnc_stats')
fare_sensitivity_results_vo=  pd.read_hdf(h5_path,key='fare_sensitivity_results_vo')
tnc_results_discount=  pd.read_hdf(h5_path,key='tnc_results_discount')
elder_demo=  pd.read_hdf(h5_path,key='elder_demo')
requests_sum=  pd.read_hdf(h5_path,key='requests_sum')
tnc_stat_summary=  pd.read_hdf(h5_path,key='tnc_stat_summary')
tnc_skim_demo=  pd.read_hdf(h5_path,key='tnc_skim_demo')
tnc_ttests=  pd.read_hdf(h5_path,key='tnc_ttests')
transit_trip_max_load_helper=  pd.read_hdf(h5_path,key='transit_trip_max_load_helper')

# Scenario Tools

## Imports

In [ ]:
from CU_POLARIS_Postprocessor.utils import select_file, bulk_update_scenario_files, copy_cases, CaseVariableData, BulkUpdateData, bulk_check_scenario_files
from CU_POLARIS_Postprocessor.utils import create_jobscript, CityKey, PolarisRunConfig, MailType
from pathlib import Path


## Creating scenarios from a base case, copying files, or updating cases

### Selecting Files

In [2]:
#case_path = select_file()
#new_case_path = select_file()
case_path = r'C:\Users\jpaul4\Box\Research\Papers\5_Mode Choice Paper\all_cases\cases'
new_case_path=r'C:\Globus\mode_choice_cases'


In [4]:
print(case_path)
print(new_case_path)

C:\Users\jpaul4\Box\Research\Papers\5_Mode Choice Paper\all_cases\cases
C:\Globus\mode_choice_cases


### Creating Scenarios from a base scenario

In [ ]:
fleet_sizes = [i for i in range(100,600,100)]
strategies =["heur",""] #for folder names
strat_key = {"heur":"default","":"DRS_Discount"} #corresponding keys for folder names (optional)

var_1 = CaseVariableData("fleet_size","SAEVFleetModel_optimization.json",["Operator_1", "Fleet_Base", "Operator_1_TNC_FLEET_SIZE"],fleet_sizes)
var_2 = CaseVariableData("strategy","SAEVFleetModel_optimization.json",["Operator_1","Fleet Strategy", "Operator_1_strategy_name"],strategies, strat_key)

new_cases = [var_1,var_2]

args = {"new_case_path":new_case_path,
        "case_path":case_path,
        "new_cases":new_cases,
        "parallel":True}

copy_cases(**args)

### Copying Cases (very quickly)
Copies a set of existing cases.

In [ ]:
#files to leave alone in the target directory if they already exist in the target directories

move_cases = True

#keep_suffixes = ['reg'] #end of folder name to move (if you only want to move some cases)
keep_suffixes = None #move all cases

args = {"new_case_path":new_case_path,
        "case_path":case_path,
        "move_cases":move_cases,
        "keep_suffixes":keep_suffixes,
        "parallel":True} #use parallel processing (faster), not necessary for small number of cases or small models

copy_cases(**args)


Moving/Copying Folders:   0%|          | 0/17 [00:00<?, ?it/s]















































































































































































































































































### Updating Models
Update all the files in each case from a single base folder (like a model base folder). Useful for "resetting" models when runs have partially completed and diverged from a base case.


In [ ]:

keep_files =['SAEVFleetModel_optimization.json','scenario_abm.json']
move_cases = False


args = {"new_case_path":new_case_path,
        "case_path":case_path,
        "move_cases":move_cases,
        "keep_files":keep_files,
        "parallel":True}

copy_cases(**args)

Moving/Copying Items: 100%|██████████| 500/500 [00:28<00:00, 17.78it/s]


## Updating scenario files universally

In [ ]:

bulk=BulkUpdateData(possible_scenario_names=["SAEVFleetModel_optimization.json"],value=10.0,target=['Operator_1','Fleet_Base','Operator_1_TNC_MAX_WAIT_TIME'])

bulk1=BulkUpdateData(possible_scenario_names=["SAEVFleetModel_optimization.json"],value=True,target=['Operator_1','Fleet_DRS','Operator_1_pool_choice_enabled'])
bulk2=BulkUpdateData(possible_scenario_names=["scenario_abm.json"],value="SAEVFleetModel_optimization.json",target=["ABM Controls","tnc_fleet_model_file"])

#updates = [bulk1,bulk2]
updates = [bulk]

for update in updates:
    bulk_update_scenario_files(update.target,update.value,new_case_path,update.scenario_name)

Updating Scenario Files: 100%|██████████| 10/10 [00:00<00:00, 104.20it/s]


### Check Scenario Files

In [ ]:

'''
checks ={"SAEVFleetModel_optimization.json":['Operator_1','Fleet_DRS','Operator_1_pool_choice_enabled'],
"scenario_abm.json":["ABM Controls","tnc_fleet_model_file"],
"SAEVFleetModel_optimization.json":["Operator_1", "Fleet_Base", "Operator_1_TNC_FLEET_SIZE"],
"SAEVFleetModel_optimization.json":["Operator_1","Fleet Strategy", "Operator_1_strategy_name"]}
'''
checks = {"SAEVFleetModel_optimization.json":['Operator_1','Fleet_Base','Operator_1_TNC_MAX_WAIT_TIME']}

folder_paths = [Path("C:/Globus/trb_cases/gsc"),Path("C:/Globus/trb_cases/atx")]



for folder_path in folder_paths:
    for scenario_names, target in checks.items():
        bulk_check_scenario_files(target, folder_path, [scenario_names])


SyntaxError: invalid syntax (3952368906.py, line 16)

### Creating a jobscript

In [ ]:
#Citykey(model_database_name(<model_name>-Suppy.sqlite),folder prefix, traffic scale factor, memory per run(gb), time per run(hours))
case_dir = r"/scratch/jpaul4/mode_choice_cases"
gsc = CityKey("greenville","gsc",1.0,16,1.5)
atx = CityKey("Austin","atx",0.25,80,3.5)

#PolarisRunConfig(container_path, run script (internal to container), threads, do abm init run, do skim run, number of abm runs, cities (CityKey objects))
run_config = PolarisRunConfig("/home/jpaul4/container/polaris_old_working.sif","run_old_polaris.py",32,False,False,7,[gsc,atx])
create_jobscript(job_name="polaris_trb_cases",mail_user="jpaul4@clemson.edu",mail_type=MailType.ALL,case_dir=case_dir,config=run_config) 